In [1]:
# necessary imports
import json
import os
import torch.nn.functional as F


In [2]:
def load_files(v_name, data_video_path):
    for videos in sorted(os.listdir(os.path.join(data_video_path, v_name))):
        if len(videos.split('_')) != 1:
            continue
        for i in json.load(open(os.path.join(data_video_path, v_name, videos))):
            json.dump([i], open(os.path.join(data_video_path, v_name, i['video']), 'w'))
        ! rm $data_video_path/$v_name/$videos


In [3]:
def create_dataset(episode_no, data_video_path, data_audio_path, type='train'):
    
    res = []
    episode_str = f'output_episode{str(episode_no).zfill(2)}'
    augmentations = ['', '_aug1', '_aug2'] if type == 'train' else ['']
    if episode_no == 18:
        augmentations = ['', '_aug1'] if type == 'train' else ['']

    for aug in augmentations:
        for (label, mode) in enumerate(['_nointeracs', '']):

            path = f'{data_video_path}/{episode_str + mode + aug}'
    
            for v_feature in sorted(os.listdir(path)):
                
                curr_file = f'{path}/{v_feature}'
                track1, track2, v = v_feature[5:].split('.')[0].split('_')

                if int(track1) > int(track2):
                    continue

                extension = '.'.join(v_feature.split('.')[1:])
                pair_loc = f'video{str(track2).zfill(3)}_{str(track1).zfill(3)}_{v}.{extension}'
                pair_file = f'{path}/{pair_loc}'
                audio_loc = f'audio_{str(track1).zfill(3)}_{str(track2).zfill(3)}_{v}_{label}.json'
                audio_file = f'{data_audio_path}/episode{str(episode_no).zfill(2)}/{audio_loc}'

                if os.path.getsize(curr_file) == 0:  continue
                if (not os.path.isfile(pair_file)) or os.path.getsize(pair_file) == 0:
                    continue
                
                feature1 = json.load(open(curr_file, 'rb'))
                feature2 = json.load(open(pair_file, 'rb'))
                feature_audio = json.load(open(audio_file)) if os.path.isfile(audio_file) else []

                if feature1 == [] or feature2 == []: continue
                assert len(feature1) == 1 and len(feature2) == 1

                if len(feature1[0]['clips']) != len(feature2[0]['clips']):
                    continue
                
                for (seg1, seg2) in zip(feature1[0]['clips'], feature2[0]['clips']):
                    res.append((seg1['features'],
                                seg2['features'],
                                feature_audio,
                                int(label)))
                                
    return res 

    

In [13]:
import random

def create_type4_negative(data_video_path, data_audio_path, start, stop, n=10000):
    res = []
    
    i = 0
    while i < n:
        ep_i = random.randint(start, stop)
        ep_j = ep_i
        while ep_j == ep_i: ep_j = random.randint(start, stop)

        mode = random.choice(['', '_nointeracs'])
        episode_stri,  episode_strj = f'episode{str(ep_i).zfill(2)}', f'episode{str(ep_j).zfill(2)}'
        path_i = f'{data_video_path}/output_{episode_stri + mode}'
        path_j = f'{data_video_path}/output_{episode_strj + mode}'

        fi = random.choice(os.listdir(path_i))
        file_i = os.path.join(path_i, fi)
        file_j = os.path.join(path_j, random.choice(os.listdir(path_j)))

        track1, track2, v = fi[5:].split('.')[0].split('_')
        label = 1 if mode == '' else 0

        audio_loc = f'audio_{str(track1).zfill(3)}_{str(track2).zfill(3)}_{v}_{label}.json'
        audio_file = f'{data_audio_path}/episode{str(ep_i).zfill(2)}/{audio_loc}'

        if os.path.isfile(audio_file):
            feature_audio = json.load(open(audio_file))
        else:
            continue
        
        featurei = json.load(open(file_i, 'rb'))
        featurej = json.load(open(file_j, 'rb'))
        if featurei == [] or featurej == []: continue

        featurei[0]['clips']
        res.append((random.choice(featurei[0]['clips']), random.choice(featurej[0]['clips']), feature_audio, label))

        i += 1
    return res

In [14]:
data_video_path = '../classification/outputs'
data_audio_path = '../data/audio_features_final_april2'

In [15]:
curr_data1 = []
for episode_no in range(1, 19):
    curr_data1 += create_dataset(episode_no, data_video_path, data_audio_path, type='train')
curr_data1 += create_type4_negative(data_video_path, data_audio_path, 1, 19, 10000)
json.dump(curr_data1, open(f'../data/dataset_with_audio/features_train.json', 'w'))

In [20]:
curr_data2 = []
for episode_no in range(19, 26):
    curr_data2 += create_dataset(episode_no, data_video_path, data_audio_path, type='test')
curr_data2 += create_type4_negative(data_video_path, data_audio_path, 1, 19, 3000)
json.dump(curr_data2, open(f'../data/dataset_with_audio/features_valid.json', 'w'))